<a href="https://colab.research.google.com/github/Pjh940114/Study_choonang/blob/main/%5Bch_0523%5D_coin(%EB%8F%99%EC%A0%84%EC%9D%B8%EC%8B%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from header.coin_preprocess import *

image, th_img = preprocessing(70)                           # 전처리 수행
if image is None: raise Exception("영상 파일 읽기 에러")

circles = find_coins(th_img)                            # 객체(회전사각형) 검출
for center, radius in circles:
    cv2.circle(image, center, radius, (0, 255, 0), 2)   # 동전 영상 원으로 표시

cv2.imshow("preprocessed image", th_img)
cv2.imshow("coin image", image)
cv2.waitKey(0)

In [ ]:
from header.coin_preprocess import *
from header.coin_utils import *                            # 기타 함수
from Common.histogram import draw_histo_hue

coin_no = 15
image, th_img = preprocessing(coin_no)                            # 전처리 수행
circles = find_coins(th_img)                     # 객체(회전사각형) 검출
coin_imgs = make_coin_img(image, circles)                # 동전 영상 생성
coin_hists = [calc_histo_hue(coin) for coin in coin_imgs] # 영상 히스토그램

for i, img in enumerate(coin_imgs):
    h, w = 200, 256
    hist_img = draw_histo_hue(coin_hists[i], (h, w, 3))    # 색상 히스토그램 표시

    merge = np.zeros((h, w+h, 3), np.uint8)
    merge[:, :w] = hist_img
    merge[:, w:] = cv2.resize(img, (h, h))
    cv2.imshow("hist&coin-" + str(i), merge)

cv2.waitKey(0)


In [ ]:
from Common.histogram import draw_histo_hue
from header.coin_preprocess import *
from header.coin_utils import *                            # 기타 함수

def onMouse(event, x, y, flags, param):
    global pre_img, hist_roi
    if event == cv2.EVENT_LBUTTONDOWN:          # 왼쪽 버튼 누르기
        for i, ((cx, cy), radius) in enumerate(circles):    # 메뉴아이콘 사각형 조회
            dx, dy = (cx - x), (cy - y)
            dist = np.sqrt(dx**2 + dy**2)     # 동전 중점좌표와 클릭좌표간 거리

            if dist < radius:
                hist_img = draw_histo_hue(coin_hists[i], (80, 128, 3))
                h, w = hist_img.shape[:2]
                hist_roi = [x, y, w, h]
                pre_img =  image[y:y + h, x:x + w].copy()
                image[y:y+h, x:x+w] = hist_img
                cv2.imshow("image", image)

    if event == cv2.EVENT_LBUTTONUP:            # 왼쪽 버튼 떼기
        x, y, w, h =  hist_roi
        image[y:y+h, x:x+w] = pre_img
        cv2.imshow("image", image)

coin_no = 15
image, th_img = preprocessing(coin_no)                            # 전처리 수행
circles = find_coins(th_img)                    # 객체 검출

coin_imgs = make_coin_img(image, circles)                # 동전 영상 생성
coin_hists = [calc_histo_hue(coin) for coin in coin_imgs] # 각 동전영상 히스토그램

for center, radius in circles:
    cv2.circle(image, center, radius, (0, 255, 0), 2)

cv2.imshow("image", image)
cv2.setMouseCallback("image", onMouse)
cv2.waitKey(0)

In [ ]:
from header.coin_preprocess import *
from header.coin_utils import *                            # 기타 함수
from Common.histogram import draw_histo_hue
import matplotlib.pyplot as plt

no, max_no = 50, 100

while True:
    gray, image = preprocessing(no)                             # 전처리 수행
    if image is None:
        print("{0:02d}.jpg: 영상 파일이 없습니다.".format(no))
        if no < 0 : no: no = max_no
        no = no + 1
        if no >= max_no: no = 0
        continue

    image, th_img = preprocessing(no)                            # 전처리 수행
    circles = find_coins(th_img)                     # 객체(회전사각형) 검출
    coin_imgs = make_coin_img(image, circles)                # 동전 영상 생성
    coin_hists = [calc_histo_hue(coin) for coin in coin_imgs] # 영상 히스토그램

    merge = np.zeros((200, 456, 3), np.uint8)
    n = int(np.ceil(len(coin_imgs)/4))
    merges = cv2.repeat(merge, n, 4)
    for i, img in enumerate(coin_imgs):
        hist_img = draw_histo_hue(coin_hists[i], (200, 256, 3))    # 색상 히스토그램 표시
        h, w = hist_img.shape[:2]
        merge[:, :w] = hist_img
        merge[:, w:] = cv2.resize(img, (h, h))
        x, y = i%4 , i//4
        y, x = np.multiply( (y, x), merge.shape[:2])
        merges[y:y+h, x:x+w+200] = merge

    cv2.imshow("hist- "+ str(no) ,merges)
    cv2.moveWindow("hist- "+ str(no), -2000,400)

    key = cv2.waitKeyEx(0)  # 키 이벤트 대기
    cv2.destroyAllWindows()
    if key == 2621440:
        no = no + 1  # 아래쪽 화살표 키이면 다음 영상
    elif key == 2490368:
        no = no - 1  # 윗쪽 화살표 키이면 이전 영상
    elif key == 32 or key == 27:
        break  # 프로그램 종료 조건

In [ ]:
from header.coin_preprocess import *
from header.coin_utils import *                            # 기타 함수
from Common.utils import put_string

coin_no = int(input("동전 영상 번호: "))
image, th_img = preprocessing(coin_no)                              # 전처리 수행
circles = find_coins(th_img)                     # 객체(회전사각형) 검출
coin_imgs = make_coin_img(image, circles)                  # 동전 영상 생성
coin_hists= [calc_histo_hue(coin) for coin in coin_imgs]   # 동전 영상 히스토그램

groups = grouping(coin_hists)                              # 동전 영상 그룹 분리
ncoins = classify_coins(circles, groups)                   # 동전 인식

coin_value = np.array([10, 50, 100, 500])                             # 동전 금액
for i in range(4):
    print("%3d원: %3d개" % (coin_value[i], ncoins[i]))

total = sum(coin_value * ncoins )           # 동전금액* 동전별 개수
str = "Total coin: {:,} Won".format(total)            # 계산된 금액 문자열
print(str)                                                 # 콘솔창에 출력
put_string(image, str, (650, 50), '', (0,230,0))

## 동전 객체에 정보(반지름, 금액) 표시
color = [(0, 0, 250), (255, 255, 0), (0, 250, 0), (250, 0, 255)]  # 동전별 색상
for i, (c, r) in enumerate(circles):
    cv2.circle(image, c, r, color[groups[i]], 2)
    put_string(image, i, (c[0] - 15, c[1] - 10), '', color[2])  # 검출 순번과 동전 반지 표시
    put_string(image, r, (c[0], c[1] + 15), '', color[3])

cv2.imshow("result image", image)
key = cv2.waitKey(0)  # 키 이벤트 대기
